In [ ]:
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
GET_TIFF_URL = "https://resultados.gob.ar/backend-difu/scope/data/getTiff/{}"

In [ ]:
df_mesas = pd.read_csv('data/mesas_por_localidad.csv', encoding='latin1')
print(len(df_mesas))

In [ ]:
# distrito_nombre = "Ciudad Autónoma de Buenos Aires"
# mesas_id_distrito = df_mesas[df_mesas['distrito_nombre'] == distrito_nombre]['identificador_unico_mesa'].values.tolist()
# len(mesas_id_distrito)

In [ ]:
mesas_id = df_mesas['identificador_unico_mesa'].values.tolist()
distritos = df_mesas['distrito_nombre'].values.tolist()

In [ ]:

from pathlib import Path
import shutil
save_path = "data/telegramas/"

shutil.rmtree(save_path, ignore_errors=True)

for distrito in distritos:
    Path(save_path + distrito + "/").mkdir(parents=True, exist_ok=True)

In [ ]:
# ! pip install pqdm

In [ ]:
from urllib.request import urlopen
import json
import base64
from os.path import isfile

In [ ]:
# tiffs_not_found = []

# for mesa_id in tqdm(mesas_id_distrito):
#     image_save_path = f"{save_path}{mesa_id}.tiff"
    
#     if not isfile(image_save_path):
#         # print(GET_TIFF_URL.format(mesa_id))
#         response = urlopen(GET_TIFF_URL.format(mesa_id)).read().decode("utf-8")
#         if len(response) > 0:
#             response = json.loads(response)

#             binary_data = base64.b64decode(response['encodingBinary'])
            
#             with open(image_save_path, "wb") as tiff_file:
#                 tiff_file.write(binary_data)
#         else:
#             tiffs_not_found.append(mesa_id)

In [ ]:
from pqdm.threads import pqdm

def download_telegrama(item):
    mesa_id = item[0]
    distrito = item[1]

    image_save_path = save_path + distrito + '/' + mesa_id + '.tiff'
    
    
    if not isfile(image_save_path):
        response = urlopen(GET_TIFF_URL.format(mesa_id)).read().decode("utf-8")
        if len(response) > 0:
            response = json.loads(response)

            binary_data = base64.b64decode(response['encodingBinary'])
            
            with open(image_save_path, "wb") as tiff_file:
                tiff_file.write(binary_data)
        else:
            tiffs_not_found.append(mesa_id)

            return mesa_id


tiffs_not_found = []

tiffs_not_found_result = pqdm(zip(mesas_id, distritos), download_telegrama, n_jobs=6)

In [ ]:
assert len(tiffs_not_found) == len([tiff for tiff in tiffs_not_found_result if tiff is not None])
len(tiffs_not_found)

In [ ]:
with open(save_path + 'tiffs_not_found.txt', 'wt') as f:
    for mesa_id in tiffs_not_found:
        f.write(f"{mesa_id}\n")